In [9]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import NN_Lib
from scipy.special import expit

in_dim = 785
hidden_dim = 100
out_dim = 10
num_epochs = 2
eta = 0.1 # Learning rate. You might try different rates (e.g. 0.001, 0.01, 0.1) to maximize the accuracy

def load_data(file_name):
    data_file = np.loadtxt(file_name, delimiter=',')
    dataset = np.insert(data_file[:, np.arange(1, in_dim)]/255, 0, 1, axis=1)
    data_labels = data_file[:, 0]
    return dataset, data_labels

In [13]:
# Initialize weight matrices with random weights
weight_1 = np.random.uniform(-0.05,0.05,(in_dim,hidden_dim)) # (785, 100)
weight_2 = np.random.uniform(-0.05,0.05,(hidden_dim+1,out_dim)) # (101, 10)
# matrix to store the activation h1...hk 
hl_input = np.zeros((1,hidden_dim+1)) # 1 101: [1 0 0 ...]
hl_input[0,0] = 1

In [6]:
# Load Training and Test Sets :
print("\nLoading Training Set")
train_data, train_labels = load_data('data/train.csv')
print("\nLoading Test Set\n")
test_data, test_labels = load_data('data/test.csv')

arr_train_acc = []
arr_test_acc = []


Loading Training Set

Loading Test Set



In [32]:
for i in range(1, num_epochs+1):
    
    pred_train_labels = get_predictions(train_data, weight_1, weight_2)  # Test network on training set and get training accuracy
    curr_accu = accuracy_score(train_labels, pred_train_labels)

    print("Epoch " + str(i) + " :\tTraining Set Accuracy = " + str(curr_accu))

    pred_test_labels = get_predictions(test_data, weight_1, weight_2)  # Test network on test set and get accuracy on test set
    test_accu = accuracy_score(test_labels, pred_test_labels)
    print("\t\tTest Set Accuracy = " + str(test_accu))

    weight_1, weight_2 = train(train_data, train_labels, weight_1, weight_2)    # Train the network

    arr_train_acc.append(curr_accu)
    arr_test_acc.append(test_accu)

Epoch 1 :	Training Set Accuracy = 0.11236666666666667
		Test Set Accuracy = 0.1135
Epoch 2 :	Training Set Accuracy = 0.11236666666666667
		Test Set Accuracy = 0.1135
Epoch 3 :	Training Set Accuracy = 0.11236666666666667
		Test Set Accuracy = 0.1135
Epoch 4 :	Training Set Accuracy = 0.11236666666666667
		Test Set Accuracy = 0.1135
Epoch 5 :	Training Set Accuracy = 0.11236666666666667
		Test Set Accuracy = 0.1135


In [61]:
def weight_update(feature, label, weight_i2h, weight_h2o): 
    # """
	# Update the weights for a train feature.
		# Inputs:
			# feature: feature vector (ndarray) of a data point with 785 dimensions. Here, the feature represents a handwritten digit 
			         # of a 28x28 pixel grayscale image, which is flattened into a 785-dimensional vector (include bias)
			# label: Actual label of the train feature 
			# weight_i2h: current weights with shape (in_dim x hidden_dim) from input (feature vector) to hidden layer
			# weight_h2o: current weights with shape (hidden_dim x out_dim) from hidden layer to output (digit number 0-9)
		# Return: updated weight
	# """
	######## feed forward ##################
	#"""compute activations at hidden layer"""
    scores_hl = np.dot(feature.reshape(1, in_dim), weight_i2h)
    sig_hl = expit(scores_hl)
    hl_input[0,1:] = sig_hl
	
	#"""compute activations at output layer"""
    scores_ol = np.dot(hl_input, weight_h2o)
    sig_ol = expit(scores_ol)    
	
	####### Update weights ##########
	#"*** YOUR CODE HERE ***"
    print(weight_i2h.shape)
    print(sig_hl.shape)
    predicted_label = np.argmax(sig_ol)
    
    tx = np.zeros(10)
    tx[int(label)] = 1
    yx = np.zeros(10)
    yx[predicted_label] = 1
    gx = np.array([1 / 2 * (tx - yx) ** 2])
    
    print(np.dot(hl_input.reshape(hidden_dim + 1, 1), gx).shape)
    
    new_weight_h20 = weight_h2o - eta * np.dot(hl_input.reshape(hidden_dim + 1, 1), gx)
    errorInInput = 1 / 2 * (new_weight_h20 - weight_h2o) ** 2
    weight_i2h -= eta * np.dot(feature.reshape(1, in_dim), errorInHidden)
    print(weight_i2h.shape)

#     feature = feature.reshape(in_dim, 1)
# 	weight_i2h = weight_i2h + eta * feature.dot(gx)
    
#     weight_h2o += eta * np.dot(hl_input
   
      
    return weight_i2h, weight_h2o
weight_update(train_data[0, :], train_labels[0], weight_1, weight_2)

(785, 100)
(1, 100)
(101, 10)


ValueError: shapes (1,785) and (101,10) not aligned: 785 (dim 1) != 101 (dim 0)

In [15]:
def train(train_set, labels, weight_i2h, weight_h2o):
    for i in range(0, train_set.shape[0]):        
        weight_i2h, weight_h2o = weight_update(train_set[i, :], labels[i], weight_i2h, weight_h2o)        
    return weight_i2h, weight_h2o

In [31]:
def get_predictions(dataset, weight_i2h, weight_h2o):
	# """
	# Calculates the predicted label for each feature in dataset.
		# Inputs:
			# dataset: a set of feature vectors with shape  
			# weight_i2h: current weights with shape (in_dim x hidden_dim) from input (feature vector) to hidden layer
			# weight_h2o: current weights with shape (hidden_dim x out_dim) from hidden layer to output (digit number 0-9)
		# Return: list (or ndarray) of predicted labels from given dataset
	# """
	# """
	# Hint: follow the feed forward step above (from lines 28-35) to compute activations at output layer. Then, find the label
	# that returns highest value of activation.
	# """
	# "*** YOUR CODE HERE ***"
    ######## feed forward ##################
	#"""compute activations at hidden layer"""
    
    scores_hl = np.dot(dataset, weight_i2h)
    sig_hl = expit(scores_hl)
    hl_inputs = np.zeros((dataset.shape[0],hidden_dim+1))
    hl_inputs[:, 0] = 1
    hl_inputs[:, 1:] = sig_hl
	
	#"""compute activations at output layer"""
    scores_ol = np.dot(hl_inputs, weight_h2o)
    sig_ol = np.argmax(expit(scores_ol), axis = 1)
    return sig_ol

In [21]:
print(train_data.shape)
print(test_data.shape)
print(weight_1.shape)
print(weight_2.shape)
in_dim = 785
hidden_dim = 100
out_dim = 10

(60000, 785)
(10000, 785)
(785, 100)
(101, 10)


In [34]:
a = np.zeros(10)
print(a.shape)
a = a.reshape(10, 1)
print(a.shape)

(10,)
(10, 1)
